# Project 3 - Group 7
Barbara MacGregor, Matt Russell, Amanda Enstad and Chi Tran

## Import dependencies

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import sqlite3
import numpy as np

## Extract
### Read in the raw excel files

In [2]:
beerCsv = '../Resources/Data/beers.csv'
brewCsv = '../Resources/Data/breweries.csv'

In [3]:
beerDf = pd.read_csv(beerCsv)
beerDf = beerDf.drop(['Unnamed: 0'],axis = 1)
beerDf.head()

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [4]:
brewDf = pd.read_csv(brewCsv)
brewDf.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


## Transform
### Rename columns

In [5]:
# Rename columns in beerDf
# abv: The alcoholic content by volume with 0 being no alcohol and 1 being pure alcohol.
# ibu: International bittering units, which describe how bitter a drink is.
# id: beer unique id
# brewery_id: Unique identifier for brewery that produces this beer; can use to join with brewery info.

beerDf = beerDf.rename(columns={"id": "beer_id", "name": "beer_name", "style": "beer_style" })
colNames = ['beer_id','beer_name','beer_style','brewery_id','abv','ibu','ounces']
beerDf = beerDf.reindex(columns=colNames)
beerDf.head()

,beer_id,beer_name,beer_style,brewery_id,abv,ibu,ounces
0,1436,Pub Beer,American Pale Lager,408,0.050,NaN,12.0
1,2265,Devil's Cup,American Pale Ale (APA),177,0.066,NaN,12.0
2,2264,Rise of the Phoenix,American IPA,177,0.071,NaN,12.0
3,2263,Sinister,American Double / Imperial IPA,177,0.090,NaN,12.0
4,2262,Sex and Candy,American IPA,177,0.075,NaN,12.0


In [6]:
# Rename columns in brewDf
brewDf = brewDf.rename(columns={"Unnamed: 0": "brewery_id", "name": "brewery_name" })
brewDf.head()

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


### Visual data inspections:
Notes: running cells in this section is optional
#### Observations:
* there are "Nan" entries in abv and ibu 
* two entries for beer style per beer: American Double / Imperial IPA (important for filtering)
* 558 unique brewery_id in beerDf but 551 unique brewery_id in brewDf

In [ ]:
# print out all of unique values in each columns in beerDf dataframe
# visually inspect to identify any NA or inconsistencies

colNames = list(beerDf.columns)
for col in colNames:
    print(col)
    print(beerDf[col].unique())
#     print(*beerDf[col].unique(),sep ='\n')
    print('Number of element of Unique Values: ')
    print(len(beerDf[col].unique()))
    print('-----------')


In [ ]:
# print out all of unique values in each columns in breweries Dataframe
# visually inspect to identify any NA or inconsistencies 

colNames = list(brewDf.columns)
for col in colNames:
    print(col)
    print(brewDf[col].unique())
#     print(*beerDf[col].unique(),sep ='\n')
    print('Number of element of Unique Values: ')
    print(len(brewDf[col].unique()))
    print('-----------')

#### Data Cleaning:

In [ ]:
# Do we want to remove brewery_id in beerDf but not in brewDf
# Are we using both tables?

# beerDf = beerDf[beerDf["brewery_id"].isin(brewDf["brewery_id"])]

## EXPORT
export clean csv for sqlite file

In [7]:
beerDf.to_csv('beers.csv', index=False,)
brewDf.to_csv('breweries.csv', index=False,)

## Create SQLITE
create sqlLite file

In [8]:
#  try sqlite3
conn = sqlite3.connect('beersDb.sqlite') 
c = conn.cursor()

In [9]:
# colNames = ['beer_id','beer_name','beer_style','brewery_id','abv','ibu','ounces']
c.execute('''CREATE TABLE beers
             ([beer_id] INTEGER PRIMARY KEY,[beer_name] text, [beer_style] text,[brewery_id] INTERGER, [abv] float ,[ibu] float ,[ounces] float )''')

c.execute('''CREATE TABLE breweries
             ([brewery_id] INTEGER PRIMARY KEY,[brewery_name] text, [city] text,[state] text)''')

conn.commit()

In [10]:
beerDf = pd.read_csv('beers.csv')
brewDf = pd.read_csv('breweries.csv')

In [ ]:
# Insert the values from the csv file into the table 'beers'
beerDf.to_sql('beers', conn, if_exists='append', index = False)  

In [ ]:
# Insert the values from the csv file into the table 'breweries'
brewDf.to_sql('breweries', conn, if_exists='append', index = False)  

###  Check sqlite database

In [ ]:
# Check to see if db is created properly
engine = create_engine("sqlite:///beersDb.sqlite")

In [ ]:
# Declare a Base using `automap_base()`
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
# Print all of the classes mapped to the Base
Base.classes.keys()

In [ ]:
pd.read_sql_query('select * from beers', con=engine)

In [ ]:
pd.read_sql_query('select * from breweries', con=engine)